In [101]:
# Item sames problem from the rel-hm dataset
# on relbench: https://relbench.stanford.edu/datasets/rel-hm/#item-sales

# 1) https://demo.kurve.ai
# 2) create graph of rel-hm data
# 3) assign article as parent node and transactions as the label node with depth 4

In [102]:
import pandas as pd
from torch_frame.utils import infer_df_stype
import catboost
from sklearn import metrics


In [103]:
train_path = 'https://kurve-customers.s3.amazonaws.com/4e1a245a-3065-4600-bb0e-a92e06ee835c/13/output/hm_item_sales_train'

In [104]:
df = pd.read_parquet(train_path)
#df = pd.concat([df, pd.read_parquet(train_path)])

In [107]:
df.head()

,arti_article_id,arti_product_code,arti_prod_name,arti_product_type_no,arti_product_type_name,arti_product_group_name,arti_graphical_appearance_no,arti_graphical_appearance_name,arti_colour_group_code,arti_colour_group_name,...,tran_3dv4_change,tran_4dv7_change,tran_7dv14_change,tran_14dv30_change,tran_30dv60_change,tran_60dv90_change,tran_90dv180_change,tran_180dv365_change,tran_365dv730_change,tran_price_label
0,1,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,0.000000,1.000000,0.333333,0.631579,0.395833,0.347826,0.489362,0.661972,1.000000,0.025373
1,6,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,0.800000,0.689655,0.527273,0.662651,0.367257,0.693252,0.469741,0.285362,0.998768,0.159542
2,8,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,0.726027,0.737374,0.556180,0.931937,0.624183,0.794805,0.607256,0.081691,0.996277,1.181339
3,9,111593,Support 40 den 1p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,0.767442,0.671875,0.663212,0.877273,0.709677,0.815789,0.652921,0.095051,0.996258,1.035322
4,10,111609,200 den 1p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,0.823529,0.708333,0.461538,0.753623,0.669903,0.746377,0.657143,0.140093,0.994691,0.157746


In [108]:
df.shape

(105542, 73)

In [109]:
[c for c in df.columns if 'label' in c]

['tran_price_label']

In [110]:
target = [c for c in df.columns if 'label' in c][0]

In [111]:
df[target]

0         0.025373
1         0.159542
2         1.181339
3         1.035322
4         0.157746
            ...   
105537    0.037254
105538    0.025407
105539    0.006763
105540    0.049102
105541    0.053847
Name: tran_price_label, Length: 105542, dtype: float64

In [112]:
df[target].max()

40.364915254237

In [113]:
df[target].min()

0.0009491525423728

In [114]:
# execute compute graph with cut date
# 9/14/2020

In [115]:
test = pd.read_parquet(train_path)

In [116]:
test.shape

(105542, 73)

In [117]:
df.shape

(105542, 73)

In [118]:
stypes = infer_df_stype(test)

In [119]:
features = [
    k for k,v in stypes.items()
    if (str(v) == 'numerical')# or str(v) == 'categorical')
    and 'label' not in k
    and not k.startswith('arti_')
    ]

In [120]:
features

['tran_customer_id_count',
 'tran_price_avg',
 'tran_price_sum',
 'tran_price_min',
 'tran_price_max',
 'cust_customer_id_count_sum',
 'cust_fashion_news_frequency_count_sum',
 'cust_age_avg_avg',
 'cust_age_avg_sum',
 'cust_age_avg_min',
 'cust_age_sum_avg',
 'cust_age_sum_sum',
 'cust_age_sum_min',
 'cust_age_min_avg',
 'cust_age_min_sum',
 'cust_age_min_min',
 'cust_age_max_avg',
 'cust_age_max_sum',
 'cust_age_max_min',
 'tran_num_events_1d',
 'tran_num_events_3d',
 'tran_num_events_4d',
 'tran_num_events_7d',
 'tran_num_events_14d',
 'tran_num_events_30d',
 'tran_num_events_60d',
 'tran_num_events_90d',
 'tran_num_events_180d',
 'tran_num_events_365d',
 'tran_num_events_730d',
 'tran_1dv3_change',
 'tran_3dv4_change',
 'tran_4dv7_change',
 'tran_7dv14_change',
 'tran_14dv30_change',
 'tran_30dv60_change',
 'tran_60dv90_change',
 'tran_90dv180_change',
 'tran_180dv365_change',
 'tran_365dv730_change']

In [92]:
cat =  ['arti_graphical_appearance_name',
 'arti_colour_group_name',
 'arti_perceived_colour_value_name',
 'arti_index_name',
 'arti_index_group_name',
 'arti_garment_group_name']

In [79]:
newfeat = []
for f in features:
    if str(stypes[f]) == 'categorical' and f in cat:
        newfeat.append(f)
    elif str(stypes[f]) == 'categorical':
        continue
    else:
        newfeat.append(f)

In [121]:
df[target] = df[target].apply(lambda x: x if not pd.isnull(x) else 0)
test[target] = test[target].apply(lambda x: x if not pd.isnull(x) else 0)

In [122]:
df[newfeat]

,arti_article_id,arti_product_code,arti_product_type_no,arti_graphical_appearance_name,arti_colour_group_name,arti_perceived_colour_value_name,arti_perceived_colour_master_id,arti_department_no,arti_index_name,arti_index_group_name,...,tran_1dv3_change,tran_3dv4_change,tran_4dv7_change,tran_7dv14_change,tran_14dv30_change,tran_30dv60_change,tran_60dv90_change,tran_90dv180_change,tran_180dv365_change,tran_365dv730_change
0,1,108775,253,Solid,White,Light,9,1676,Ladieswear,Ladieswear,...,NaN,0.000000,1.000000,0.333333,0.631579,0.395833,0.347826,0.489362,0.661972,1.000000
1,6,111565,304,Solid,Black,Dark,5,3608,Lingeries/Tights,Ladieswear,...,0.250000,0.800000,0.689655,0.527273,0.662651,0.367257,0.693252,0.469741,0.285362,0.998768
2,8,111586,273,Solid,Black,Dark,5,3608,Lingeries/Tights,Ladieswear,...,0.132075,0.726027,0.737374,0.556180,0.931937,0.624183,0.794805,0.607256,0.081691,0.996277
3,9,111593,304,Solid,Black,Dark,5,3608,Lingeries/Tights,Ladieswear,...,0.348485,0.767442,0.671875,0.663212,0.877273,0.709677,0.815789,0.652921,0.095051,0.996258
4,10,111609,304,Solid,Black,Dark,5,3608,Lingeries/Tights,Ladieswear,...,0.142857,0.823529,0.708333,0.461538,0.753623,0.669903,0.746377,0.657143,0.140093,0.994691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,36401,650672,91,Solid,White,Light,9,3527,Ladies Accessories,Ladieswear,...,1.000000,0.500000,0.666667,1.000000,0.375000,0.320000,0.446429,0.674699,0.230556,1.000000
105538,36418,650693,254,Stripe,Dark Blue,Dark,2,1636,Ladieswear,Ladieswear,...,NaN,NaN,NaN,0.000000,0.142857,0.368421,0.447059,0.447368,0.931373,1.000000
105539,36455,650798,265,Solid,Dark Blue,Dark,2,7814,Children Sizes 92-140,Baby/Children,...,NaN,NaN,0.000000,1.000000,1.000000,0.100000,0.208333,0.941176,0.894737,1.000000
105540,36623,651337,252,All over pattern,Grey,Medium Dusty,12,1660,Ladieswear,Ladieswear,...,0.000000,1.000000,0.500000,0.666667,0.500000,0.428571,0.518519,0.931034,0.266055,1.000000


In [123]:
#features +=    cat

In [127]:
import numpy as np
import pandas as pd
from pandas.api.types import is_object_dtype, is_categorical_dtype, is_bool_dtype
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool

# --------------------------------------------
# 0) Split once into train_full / test
# --------------------------------------------
X_train_full, X_test, y_train_full, y_test = train_test_split(
    df[features], df[target],
    test_size=0.20,
    random_state=42
)

# --------------------------------------------
# Identify categorical columns (by name)
# --------------------------------------------
cat_features = [
    col for col in X_train_full.columns
    if is_object_dtype(X_train_full[col])
       or is_categorical_dtype(X_train_full[col])
       or is_bool_dtype(X_train_full[col])
]
print(f"Categorical features: {cat_features}")

# Optional (harmless): ensure 'category' dtype for object-y cats (speeds up a bit)
for c in cat_features:
    if not is_categorical_dtype(X_train_full[c]):
        X_train_full[c] = X_train_full[c].astype('category')
        X_test[c]       = X_test[c].astype('category')

# --------------------------------------------
# 1) K-Fold CV on train_full (no log transforms)
# --------------------------------------------
k = 3
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold_maes = []
test_preds = np.zeros(len(X_test))        # ensemble predictions on test (original scale)
oof_preds  = np.zeros(len(X_train_full))  # optional OOF preds (original scale)
best_iters = []

for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train_full), 1):
    print(f"\n=== Fold {fold} ===")

    X_tr, X_va = X_train_full.iloc[idx_tr], X_train_full.iloc[idx_va]
    y_tr, y_va = y_train_full.iloc[idx_tr], y_train_full.iloc[idx_va]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
    valid_pool = Pool(X_va, y_va, cat_features=cat_features)

    mdl = CatBoostRegressor(
        # objective & metrics (L1 on original scale)
        loss_function="MAE",
        eval_metric="MAE",
        custom_metric=["MAE", "RMSE", "R2"],
        use_best_model=True,

        # capacity vs regularization
        iterations=2000,
        learning_rate=0.03,
        depth=7,
        l2_leaf_reg=10,

        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,

        # tree / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",

        # early stopping
        od_type="Iter",
        od_wait=150,

        verbose=False
    )

    mdl.fit(train_pool, eval_set=valid_pool)
    best_iters.append(mdl.best_iteration_ if mdl.best_iteration_ is not None else mdl.tree_count_)

    # ---- Fold validation (MAE on original scale) ----
    val_pred = mdl.predict(valid_pool)
    val_mae = mean_absolute_error(y_va, val_pred)
    fold_maes.append(val_mae)
    print(f"Fold {fold} validation MAE : {val_mae:.4f}")

    # ---- Accumulate ensemble predictions on the held-out test ----
    test_preds += mdl.predict(Pool(X_test, cat_features=cat_features)) / k

    # (optional) OOF predictions
    oof_preds[idx_va] = val_pred

# --------------------------------------------
# 2) CV summary and final test metric (original scale)
# --------------------------------------------
print("\n=== CV Summary ===")
print(f"Mean CV MAE : {np.mean(fold_maes):.4f} ± {np.std(fold_maes):.4f}")
print(f"Folds MAE   : {[f'{a:.4f}' for a in fold_maes]}")

test_mae = mean_absolute_error(y_test, test_preds)
print(f"\nFinal Test MAE (averaged over {k} folds): {test_mae:.4f}")

yhat_test = test_preds  # already in original target units

# --------------------------------------------
# 3) Refit on the full training data for deployment
# --------------------------------------------
final_iters = int(np.median(best_iters)) if len(best_iters) > 0 else 2000

final_mdl = CatBoostRegressor(
    loss_function="MAE",
    eval_metric="MAE",
    custom_metric=["MAE", "RMSE", "R2"],
    use_best_model=True,

    iterations=final_iters,
    learning_rate=0.03,
    depth=7,
    l2_leaf_reg=10,

    bootstrap_type="Bayesian",
    bagging_temperature=0.75,
    rsm=0.8,
    random_strength=0.5,

    feature_border_type="GreedyLogSum",
    min_data_in_leaf=20,
    boosting_type="Plain",

    od_type="Iter",
    od_wait=150,

    verbose=False
)

final_mdl.fit(
    Pool(X_train_full, y_train_full, cat_features=cat_features),
    eval_set=Pool(X_test, y_test, cat_features=cat_features)  # sanity eval on original scale
)


/var/folders/1x/vtf00f393139j34xpbgrsnrm0000gn/T/ipykernel_85329/3579608705.py:23: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  or is_categorical_dtype(X_train_full[col])


Categorical features: []

=== Fold 1 ===
Fold 1 validation MAE : 0.0492

=== Fold 2 ===
Fold 2 validation MAE : 0.0401

=== Fold 3 ===
Fold 3 validation MAE : 0.0442

=== CV Summary ===
Mean CV MAE : 0.0445 ± 0.0037
Folds MAE   : ['0.0492', '0.0401', '0.0442']

Final Test MAE (averaged over 3 folds): 0.0404


In [128]:
test['pred'] = final_mdl.predict(test[features])

In [129]:
metrics.mean_absolute_error(test[target], test['pred'])

0.03505560185574816

In [99]:
# with categoricals added
test['pred'] = mdl.predict(test[features])
metrics.mean_absolute_error(test[target], test['pred'])

0.03668415228475982

In [100]:
list(reversed(sorted(zip(mdl.feature_names_, mdl.feature_importances_),key=lambda x: x[1])))

[('tran_num_events_3d', 20.280866499194996),
 ('tran_num_events_4d', 17.90485071004516),
 ('tran_num_events_7d', 14.711981195813584),
 ('tran_price_avg', 14.510564878375018),
 ('tran_num_events_1d', 10.224390682864525),
 ('tran_price_min', 5.484093096945771),
 ('tran_num_events_14d', 3.032218074435942),
 ('tran_price_sum', 2.8592670646458624),
 ('tran_30dv60_change', 1.3812336962635152),
 ('tran_60dv90_change', 1.3545085952817753),
 ('tran_price_max', 1.2742029089036693),
 ('tran_num_events_30d', 1.0602951617978913),
 ('tran_14dv30_change', 1.0202473032351882),
 ('tran_7dv14_change', 0.8558262141483541),
 ('arti_garment_group_name', 0.5640389808995813),
 ('tran_180dv365_change', 0.4977812385349456),
 ('tran_num_events_60d', 0.3415359669856667),
 ('tran_4dv7_change', 0.33781229729819123),
 ('tran_90dv180_change', 0.33016326385859274),
 ('arti_index_name', 0.293747765064915),
 ('tran_3dv4_change', 0.2852936215901493),
 ('tran_num_events_180d', 0.2369002440191425),
 ('cust_age_avg_avg', 0